In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
# !pip3 install transformers==4.34.1

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# tokenizer = AutoTokenizer.from_pretrained('aisingapore/sealion7b', trust_remote_code = True)
# model = AutoModelForCausalLM.from_pretrained(
#     'aisingapore/sealion7b',
#     trust_remote_code = True,
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )
tokenizer = AutoTokenizer.from_pretrained('aisingapore/sealion7b', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(
    'aisingapore/sealion7b',
    trust_remote_code = True,
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sealion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████| 2/2 [04:37<00:00, 138.78s/it]


In [4]:
from tqdm import tqdm
import os
import random
torch.manual_seed(404)
random.seed(404)

In [5]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/fizik-spm/data/'
with open(data_folder+ 'fizik') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    #objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)
    print(data)
    print()

{'no': '1', 'soalan': 'Antara yang berikut, yang manakah kuantiti vektor?\nA. Tenaga\nB. Jisim\nC. Daya\nD. Laju', 'jawapan': 'C'}

{'no': '2', 'soalan': 'Manakah antara berikut mempunyai inersia yang paling kecil?\nA. 1.8 kg arnab\nB. 850 g spanar\nC. 1350 g kuali\nD. 1.25 kg mikroskop', 'jawapan': 'B'}

{'no': '3', 'soalan': 'Apakah konsep yang digunakan dalam pengukuran suhu kopi menggunakan termometer?\nA. Perolakan terma\nB. Muatan haba tentu\nC. Haba pendam tentu\nD. Keseimbangan terma', 'jawapan': 'D'}

{'no': '4', 'soalan': 'Antara hukum yang berikut, manakah menyatakan hubungan antara tekanan dan suhu pada isi padu gas malar?\nA. Hukum Boyle\nB. Hukum Charles\nC. Hukum Gay-Lussac\nD. Hukum Avogadro', 'jawapan': 'C'}

{'no': '5', 'soalan': 'Antara berikut, yang manakah merupakan ciri gelombang biasan dari kawasan air cetek ke kawasan air dalam?\nA. Sudut tuju lebih daripada sudut pembiasan.\nB. Panjang gelombang meningkat.\nC. Frekuensi berkurang.\nD. Frekuensi meningkat.', 'ja

In [6]:
arange = set(range(len(questions)))

In [7]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [8]:
i = 1
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

soalan: Manakah antara berikut mempunyai inersia yang paling kecil?
A. 1.8 kg arnab
B. 850 g spanar
C. 1350 g kuali
D. 1.25 kg mikroskop
jawapan:


In [9]:
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[128148, 249853,   1562,  15095,  19577,  20418,  17575,    337,    393,
            523,    948,  13134,  19598, 249905,      4, 249843, 249835, 249813,
         249845, 249835, 249880,  17171,    613, 139863,      4, 249850, 249835,
         249813, 249880, 249872, 249841,    346,  17233,    329,      4, 249844,
         249835, 249813, 249845, 249869, 249872, 249841,    346,    378,  59586,
              4, 249852, 249835, 249813, 249845, 249835, 249846, 249872,  17171,
          73714,   5602,  51994,      4,  64068,   3329, 249853]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
inputs.pop('token_type_ids', None)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [11]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=3,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]).split('jawapan:')[1].strip().split())

['D', 'So']


In [12]:
for i in tqdm(range(len(questions))):
    
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    inputs.pop('token_type_ids', None)
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=3,
                top_p=0.95,
                top_k=50,
                temperature=0.5,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            r = tokenizer.decode(r[0]).split('jawapan:')[-1].strip().split()
            repeat.append(r[0].replace('.', '').replace('</s>', '').split('\\')[0].split('/')[0])
    
        except Exception as e:
            print(e)
            pass
    
    questions[i]['output'] = repeat

  0%|                                                                                                                  | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_27388/2376134102.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  shots = random.sample(arange - {i}, 1)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:38<00:00,  4.91s/it]


In [13]:
import json

with open('output-1shot-aisingapore-sealion7b.json', 'w') as fopen:
    json.dump(questions, fopen)

In [14]:
def most_common(l):
    return max(set(l), key=l.count)

In [15]:
filtered = [q for q in questions if 'output' in q]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

25.0

In [16]:
repeat

['D', 'D', 'C', 'D', 'C']

In [17]:
prompt

'Contoh soalan 1\nsoalan: Antara berikut, yang manakah merupakan ciri gelombang biasan dari kawasan air cetek ke kawasan air dalam?\nA. Sudut tuju lebih daripada sudut pembiasan.\nB. Panjang gelombang meningkat.\nC. Frekuensi berkurang.\nD. Frekuensi meningkat.\njawapan: B\n\nsoalan: Tekanan atmosfera boleh diukur dengan menggunakan\nA. barometer\nB. tolok Bourdon\nC. hidrometer\nD. oximeter\njawapan:'